In [7]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE

In [8]:
df = pd.read_csv("fixed_merge1.csv")

In [9]:
# Inspect dataset
print("Dataset Info:")
print(df.info())
print("\nFirst few rows:")
print(df.head())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Columns: 20373 entries, Unnamed: 0 to cancer
dtypes: int64(20369), object(4)
memory usage: 44.3+ MB
None

First few rows:
            Unnamed: 0  ENSG00000000003  ENSG00000000419  ENSG00000000457  \
0   3-Breast-Her2-ampl                0               44               26   
1          8-Breast-WT                0               14                1   
2  10-Breast-Her2-ampl                0               16               14   
3           Breast-100                0                8                0   
4  15-Breast-Her2-ampl               17                9                4   

   ENSG00000000460  ENSG00000000938  ENSG00000000971  ENSG00000001036  \
0               81              171               34               83   
1               98               99                0              107   
2               18               25                0               62   
3               17                4  

In [10]:


# Check association between cancer_type and mutational_subclass
print("\nChecking association between cancer_type and mutational_subclass:")
contingency_table = pd.crosstab(df['cancer_type'], df['mutational_subclass'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-squared test p-value: {p}")
if p < 0.05:
    print("Strong association detected (p < 0.05). Excluding mutational_subclass to avoid leakage.")
else:
    print("Weak association (p >= 0.05). Mutational_subclass may be a useful feature.")

# Define columns to drop (non-feature columns)
columns_to_drop = ['Unnamed: 0', 'patient_id', 'cancer', 'cancer_type', 'mutational_subclass']

# Features = all columns except non-feature columns
X = df.drop(columns=columns_to_drop)
y = df["cancer_type"]

# Remove constant features
var_filter = VarianceThreshold(threshold=0.0)
X = var_filter.fit_transform(X)
print(f"Removed {len(var_filter.get_support()) - sum(var_filter.get_support())} constant features.")

# Verify all features are numeric
non_numeric_cols = pd.DataFrame(X).select_dtypes(include=['object', 'category']).columns
if len(non_numeric_cols) > 0:
    raise ValueError(f"Non-numeric feature columns detected: {non_numeric_cols}. Please encode or drop them.")

# Encode target labels (cancer_type)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Stratified split (preserves class distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42, k_neighbors=1)  # Reduced k_neighbors for small classes
X_train, y_train = smote.fit_resample(X_train, y_train)
print(f"After SMOTE, training set size: {X_train.shape}")

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Feature selection (pick top 200 features to balance informativeness and noise)
k = min(200, X.shape[1])
selector = SelectKBest(f_classif, k=k)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

# Define classifiers with class weights
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced'),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Logistic Regression": LogisticRegression(multi_class='multinomial', max_iter=1000, random_state=42, class_weight='balanced'),
    "SVM": SVC(kernel='linear', random_state=42, class_weight='balanced')
}

# Train, evaluate, and collect F1-scores
f1_scores = {}
for name, clf in classifiers.items():
    print(f"\n### Evaluating {name} ###")
    
    # Train classifier
    clf.fit(X_train, y_train)
    
    # Predictions
    y_pred = clf.predict(X_test)
    
    # Evaluation
    print(f"Classification Report for {name}:\n", 
          classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))
    print(f"Confusion Matrix for {name}:\n", 
          confusion_matrix(y_test, y_pred))
    
    # Store weighted F1-score
    f1_scores[name] = f1_score(y_test, y_pred, average='weighted')

# Print F1-scores for comparison
print("\nWeighted F1-Scores:")
for name, score in f1_scores.items():
    print(f"{name}: {score:.4f}")


Checking association between cancer_type and mutational_subclass:
Chi-squared test p-value: 1.3646478496712181e-40
Strong association detected (p < 0.05). Excluding mutational_subclass to avoid leakage.
Removed 0 constant features.
After SMOTE, training set size: (336, 20368)


C:\Users\User\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [ 6751  9867 10234 13117 13182 13286 15114 15746 15890 15908 16020 16076
 16170 16229 16275 16397 16533 16624 16642 16723 16756 16856 17031 17302
 17447 17558 17717 17862 17924 18006 18210 18351 18862 18867 19098 19183
 19235 19393 19452 19691 19726 19829 19836 19897 20078 20128 20176] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\User\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw



### Evaluating Random Forest ###
Classification Report for Random Forest:
                             precision    recall  f1-score   support

       cancer type: Breast       0.40      0.25      0.31         8
          cancer type: CRC       0.33      0.62      0.43         8
          cancer type: GBM       0.25      0.25      0.25         8
           cancer type: HC       0.75      0.55      0.63        11
cancer type: Hepatobiliary       0.00      0.00      0.00         3
         cancer type: Lung       0.23      0.25      0.24        12
     cancer type: Pancreas       0.29      0.29      0.29         7

                  accuracy                           0.35        57
                 macro avg       0.32      0.32      0.31        57
              weighted avg       0.37      0.35      0.35        57

Confusion Matrix for Random Forest:
 [[2 2 1 0 0 0 3]
 [0 5 0 0 0 2 1]
 [0 2 2 1 0 2 1]
 [0 0 1 6 0 4 0]
 [0 0 1 1 0 1 0]
 [3 4 2 0 0 3 0]
 [0 2 1 0 1 1 2]]

### Evaluating 

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Classification Report for Logistic Regression:
                             precision    recall  f1-score   support

       cancer type: Breast       0.33      0.38      0.35         8
          cancer type: CRC       0.42      0.62      0.50         8
          cancer type: GBM       0.62      0.62      0.62         8
           cancer type: HC       0.67      0.55      0.60        11
cancer type: Hepatobiliary       0.00      0.00      0.00         3
         cancer type: Lung       0.33      0.33      0.33        12
     cancer type: Pancreas       0.57      0.57      0.57         7

                  accuracy                           0.47        57
                 macro avg       0.42      0.44      0.43        57
              weighted avg       0.46      0.47      0.46        57

Confusion Matrix for Logistic Regression:
 [[3 2 0 0 0 0 3]
 [1 5 0 0 0 2 0]
 [1 0 5 0 0 2 0]
 [0 1 2 6 0 2 0]
 [0 0 0 2 0 1 0]
 [4 2 1 1 0 4 0]
 [0 2 0 0 0 1 4]]

### Evaluating SVM ###
Classification